In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model

In [5]:
train_x = []
train_y = []

for i in range(1,5001):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    train_x.append(tmp[:63])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [6]:
train_x.shape

(5000, 63, 22)

In [15]:
model = tf.keras.models.Sequential()
model.add(LSTM(units=22, activation='softmax', input_shape=(63,22), dropout=0.3, recurrent_dropout=0.3))

In [16]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=7)

In [17]:
with tf.device("/gpu:0"):
    model.fit(train_x,train_y, epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
141/141 [==============================] - 50s 347ms/step - loss: 2.9851 - accuracy: 0.2080 - val_loss: 2.8770 - val_accuracy: 0.1960
Epoch 2/4000
141/141 [==============================] - 50s 354ms/step - loss: 2.7405 - accuracy: 0.3002 - val_loss: 2.6003 - val_accuracy: 0.3260
Epoch 3/4000
141/141 [==============================] - 50s 351ms/step - loss: 2.4081 - accuracy: 0.3329 - val_loss: 2.2661 - val_accuracy: 0.3360
Epoch 4/4000
141/141 [==============================] - 49s 348ms/step - loss: 2.1716 - accuracy: 0.3840 - val_loss: 2.1701 - val_accuracy: 0.3660
Epoch 5/4000
141/141 [==============================] - 49s 349ms/step - loss: 2.0976 - accuracy: 0.3940 - val_loss: 2.1027 - val_accuracy: 0.3660
Epoch 6/4000
141/141 [==============================] - 50s 353ms/step - loss: 2.0355 - accuracy: 0.3976 - val_loss: 2.0558 - val_accuracy: 0.3720
Epoch 7/4000
141/141 [==============================] - 49s 351ms/step - loss: 1.9933 - accuracy: 0.4007 - val_loss: 2

In [18]:
model.save('64.h5')

In [21]:
import functools
top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

dilla.wav


In [24]:
truth_loop = [7, 22, 5, 22, 13, 22, 5, 5, 5, 22, 7, 22, 13, 7, 5, 22]

prd = [0, 21, 21, 21, 15, 21, 21, 21, 5,  21, 0 , 21, 1 , 0,  21, 21,\
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0 , 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 0 , 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 10, 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 12, 21, 0,  21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 15, 21, 21, 21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 0,  21, 1 , 10, 21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 0,  21, 7,  21, 0,  21, 15, 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1,  0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1,  0,  21, 21, \
       0, 21, 21, 21, 12, 21, 21, 21, 5,  21, 0,  21, 1,  0,  21, 21]

ground_truth = [] # loop * 12


In [25]:
prd_input = np.zeros((prd.__len__(), 22))

i = 0
for p in prd:
    prd_input[i][int(p)] = 1
    i += 1


In [26]:
prd_input.shape

(192, 22)

In [28]:
change = []
for i in range(125):
    change.append(np.argmax(model.predict(np.expand_dims(prd_input[i:i+63], axis=0))))

In [29]:
change

[21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21]

In [51]:
cnt = 0
asdf = 0
for i in train_y:
    cnt += 1
    if np.argmax(i) == 22:
        asdf += 1

In [53]:
asdf

903